In [16]:
import scipy.fft
import scipy.io
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import time
import urllib.request
import math
import pyproj
import pandas as pd
import urllib
import re
import datetime as dt

from nptdms import TdmsFile
import h5py
import scipy.io
import glob
import os

# Convert Files to Matlab

## Convert TDMS Files

In [ ]:
import scipy.io
import glob
import os
import datetime as dt
from nptdms import TdmsFile
import numpy as np

files = sorted(glob.glob(r'../../Data/MarineMammals/Original/*.tdms'))
outputLocation = r'../../Data/MarineMammals/MatlabFormat'
vendor             = 'Slx'
outputRecordLength = 15
for file in files:

    print(file)
    dateTime            = dt.datetime.strptime(file[file.rfind('UTC_')+4:-9],'%Y%m%d_%H%M%S')
    thisTDMS            = TdmsFile.read(file)
    group               = thisTDMS['Measurement']
    props               = thisTDMS.properties

    samplingFrequency   = props['SamplingFrequency[Hz]']
    spatialSampling     = props['SpatialResolution[m]' ]
    gaugeLength         = props['GaugeLength'          ]
    measureLength       = props['MeasureLength[m]'     ]
    startTime           = props['GPSTimeStamp'         ]
    
    cable = 'South'
    
    acousticData = np.asarray( [group[channel].data for channel in group] )
    
    print(acousticData)
    print(acousticData.shape,file[:-4]+str(int(samplingFrequency))+'_'+str(int(gaugeLength))+'_'+str(int(spatialSampling))+'.mat')
    outputFileName =  vendor                               + '_' \
                    + cable                                + '_' \
                    + dateTime.strftime('%Y%m%d%H%M%S') + '_' \
                    + str(int(gaugeLength      ))          + '_' \
                    + str(int(spatialSampling  ))          + '_' \
                    + str(int(samplingFrequency))
    print(outputFileName+'.mat')
    outputFilePath = outputLocation + '\\' + outputFileName
    print(outputFilePath)
    scipy.io.savemat(outputFilePath+'.mat',{'data':acousticData})
    
    thisTDMS.close()
    

## Convert h5 files

The output record length is the number of seconds a file will be split into so that they can be saved and processed effectively as .mat files. That value is currently set to 15 seconds, however some larger files may need to be split into smaller chunks to actually be able to save them.
In the example data provided for the Marine Mammals, only the 'North-C3-HF-P1kHz-GL30m-Sp2m_2021-11-02T114145Z.h5' file was set to 5 seconds.

In [ ]:
import scipy.io
import glob
import os
import datetime as dt
import numpy as np
import h5py

files = sorted(glob.glob(r'../../Data/MarineMammals/Original/*.h5'))
outputLocation = r'../../Data/MarineMammals/MatlabFormat'

vendor             = 'Opt'
outputRecordLength = 15 
for file in files:

    print(file)

    timeStamp         = file[file.rfind("_2021-")+1:file.rfind("_2021-")+18]
    dateTime          = dt.datetime.strptime(timeStamp, '%Y-%m-%dT%H%M%S')
    newDateTime       = dateTime.strftime('%Y%m%d%H%M%S')

    cable = 'North'
    if file.find('South') >= 0:
        cable= 'South'
    
    print(vendor,cable,dateTime.strftime('%Y%m%d%H%M%S') )
    
    if True:
        
        fp                = h5py.File(file,'r')

        gaugeLength       = fp['Acquisition'].attrs['GaugeLength']
        samplingInterval  = fp['Acquisition'].attrs['SpatialSamplingInterval']
        samplingFrequency = fp['Acquisition']['Raw[0]'].attrs['OutputDataRate']

        print(0,60*int(samplingFrequency))
        print(int(outputRecordLength*samplingFrequency))
        
        for startSample in range(0,60*int(samplingFrequency),int(outputRecordLength*samplingFrequency)):
            stopSample = startSample + int(outputRecordLength*samplingFrequency)

            newDateTime       = dateTime + dt.timedelta(seconds=startSample/samplingFrequency)

            rawData           = np.asarray( fp['Acquisition']['Raw[0]']['RawData'][:,startSample:stopSample] )
            if rawData.size < 1:
                continue

            outputFileName =  vendor                               + '_' \
                            + cable                                + '_' \
                            + newDateTime.strftime('%Y%m%d%H%M%S') + '_' \
                            + str(int(gaugeLength      ))          + '_' \
                            + str(int(samplingInterval ))          + '_' \
                            + str(int(samplingFrequency))
            print(outputFileName+'.mat')
            outputFilePath = outputLocation + '\\' + outputFileName
            print(outputFilePath+'.mat')
            print(rawData)

            scipy.io.savemat(outputFilePath+'.mat',{'data':rawData})
    File 
        fp.close()

# Convert TDMS with Vessel MMSI

In [ ]:
import scipy.io
import glob
import os
import datetime as dt
from nptdms import TdmsFile
import numpy as np
import pandas as pd

files = sorted(glob.glob(r'../../Data/Vessels/Original/*.tdms'))
outputLocation = r'../../Data/Vessels/MatlabFormat'
vendor             = 'Slx'
outputRecordLength = 15

mmsiList = []
fileList = []
for file in files:

    print(file)
    mmsi = file[file.rfind('Original')+9:file.rfind('Original')+18]
    dateTime            = dt.datetime.strptime(file[file.rfind('UTC_')+4:-9],'%Y%m%d_%H%M%S')
    thisTDMS            = TdmsFile.read(file)
    group               = thisTDMS['Measurement']
    props               = thisTDMS.properties

    samplingFrequency   = props['SamplingFrequency[Hz]']
    spatialSampling     = props['SpatialResolution[m]' ]
    gaugeLength         = props['GaugeLength'          ]
    measureLength       = props['MeasureLength[m]'     ]
    startTime           = props['GPSTimeStamp'         ]
    
    cable = 'South'
    
    acousticData = np.asarray( [group[channel].data for channel in group] )
    
    print(acousticData)
    print(acousticData.shape,file[:-4]+str(int(samplingFrequency))+'_'+str(int(gaugeLength))+'_'+str(int(spatialSampling))+'.mat')
    outputFileName =  vendor                               + '_' \
                    + cable                                + '_' \
                    + dateTime.strftime('%Y%m%d%H%M%S') + '_' \
                    + str(int(gaugeLength      ))          + '_' \
                    + str(int(spatialSampling  ))          + '_' \
                    + str(int(samplingFrequency))
    print(outputFileName+'.mat')
    outputFilePath = outputLocation + '\\' + outputFileName +'.mat'
    mmsiList.append(mmsi)
    fileList.append((outputFileName)+'.mat')
    print(outputFilePath)
    scipy.io.savemat(outputFilePath,{'data':acousticData})
    
    thisTDMS.close()
vesselTable = {
    'MMSI':mmsiList,
    'FileName':fileList
}
df = pd.DataFrame(vesselTable)
df.to_csv(r'../../Documentation/Catalogues/VesselMMSI.csv')